### 기본 시뮬레이션

In [1]:
import gym

In [2]:
env = gym.make("CartPole-v0")

In [3]:
env.reset()

array([-0.02499827,  0.00660312,  0.01606982, -0.0115705 ])

In [4]:
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample())

/home/amelia/anaconda3/envs/universe/lib/python3.8/site-packages/gym/logger.py:34: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


#### 간단한 환경 목록 확인 

In [11]:
from gym import envs
print(envs.registry.all())

dict_values([EnvSpec(Copy-v0), EnvSpec(RepeatCopy-v0), EnvSpec(ReversedAddition-v0), EnvSpec(ReversedAddition3-v0), EnvSpec(DuplicatedInput-v0), EnvSpec(Reverse-v0), EnvSpec(CartPole-v0), EnvSpec(CartPole-v1), EnvSpec(MountainCar-v0), EnvSpec(MountainCarContinuous-v0), EnvSpec(Pendulum-v0), EnvSpec(Acrobot-v1), EnvSpec(LunarLander-v2), EnvSpec(LunarLanderContinuous-v2), EnvSpec(BipedalWalker-v3), EnvSpec(BipedalWalkerHardcore-v3), EnvSpec(CarRacing-v0), EnvSpec(Blackjack-v0), EnvSpec(KellyCoinflip-v0), EnvSpec(KellyCoinflipGeneralized-v0), EnvSpec(FrozenLake-v1), EnvSpec(FrozenLake8x8-v1), EnvSpec(CliffWalking-v0), EnvSpec(NChain-v0), EnvSpec(Roulette-v0), EnvSpec(Taxi-v3), EnvSpec(GuessingGame-v0), EnvSpec(HotterColder-v0), EnvSpec(Reacher-v2), EnvSpec(Pusher-v2), EnvSpec(Thrower-v2), EnvSpec(Striker-v2), EnvSpec(InvertedPendulum-v2), EnvSpec(InvertedDoublePendulum-v2), EnvSpec(HalfCheetah-v2), EnvSpec(HalfCheetah-v3), EnvSpec(Hopper-v2), EnvSpec(Hopper-v3), EnvSpec(Swimmer-v2), EnvSp

#### Car Racing environment 

In [1]:
import gym 
env = gym.make('CarRacing-v0')
env.reset()
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample())

Track generation: 1150..1442 -> 292-tiles track


### 로봇 보행 훈련시키기

In [5]:
import gym
env = gym.make("BipedalWalker-v3") #v2 deprecated

In [6]:
for episode in range(100):
    observation = env.reset()

In [9]:
for i in range(1000):
    env.render()
    action = env.action_space.sample() #행동 공간에서 무작위로 임의의 해동을 선택
    observation, reward, done, info = env.step(action)
    if done:
        print("{} timesteps taken for the Episode".format(i+1))
        break

600 timesteps taken for the Episode


## OpenAI Universe
OpenGym을 확장한 것
### 비디오 게임 봇 생성하기 

In [2]:
import gym #0.9.5 >= gym "gym.benchmarks" deprecated
import universe
import random

/home/amelia/github/OpenAIGymTutorial/universe/universe/runtimes/__init__.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  spec = yaml.load(f)


In [8]:
env = gym.make("flashgames.NeonRace-v0")
env.configure(remotes=1) # create local docker container
observation_n=env.reset()

[2021-09-07 18:43:45,878] Making new env: flashgames.NeonRace-v0
[2021-09-07 18:43:45,895] Ports used: dict_keys([])
[2021-09-07 18:43:45,896] [0] Creating container: image=quay.io/openai/universe.flashgames:0.20.28. Run the same thing by hand as: docker run -p 5900:5900 -p 15900:15900 --privileged --cap-add SYS_ADMIN --ipc host quay.io/openai/universe.flashgames:0.20.28
[2021-09-07 18:43:45,900] Image quay.io/openai/universe.flashgames:0.20.28 not present locally; pulling


APIError: 500 Server Error: Internal Server Error ("b'{"message":"unauthorized: access to the requested resource is not authorized"}'")

2018 이후 최근 업데이트가 없음 openai/universe=> deprecated

In [9]:
left = [("KeyEvent", "ArrowUp", True), ("KeyEvent", "ArrowLeft", True), ("KeyEvent", "ArrowRight", False)]
right = [("KeyEvent", "ArrowUp", True), ("KeyEvent", "ArrowLeft", False), ("KeyEvent", "ArrowRight", True)]
forward = [("KeyEvent", "ArrowUp", True), ("KeyEvent", "ArrowLeft", False), ("KeyEvent", "ArrowRight", False),("KeyEvent", 'n', True)]

In [10]:
# We use turn variable for deciding whether to turn or not
turn = 0

# We store all the rewards in rewards list
rewards = []

# we will use buffer as some threshold
buffer_size=100

#we will initially set action as forward, which just move the car forward without any turn
action = forward

In [ ]:
while True:
    turn -= 1
    # Let's say initially we take no turn and move forward
    # We will check value of turn, it it is less than 0
    # then there is no nessity for turning and we just move forward
    
    if turn <= 0:
        action = forward
        turn = 0
        action_n = [action for ob in observation_n]
        observation_n, reward_n, done_n, info = env.step(action_n) #자동차 상태, 보상, TF, for debugging
        
        rewards += [reward_n[0]]
        if len(rewards) >= buffer_size:
            mean = sum(rewards)/len(rewards)
            
            if mean == 0:
                turn = 20
                if random.random() < 0.5: #정책함수에 의한 임의의 정책
                    action = right
                else: 
                    action = left
            
            rewards = []
        env.render()